In [123]:
import pandas as pd
import numpy as np
from analysis.analysis import create_analysis
from analysis.pipeline import create_pipeline
from analysis.stages import BooleanSummary, ClassifyColumns, FormatData, NumericalSummary, CategoricalSummary, Correlation





In [124]:
test_data = {
    "scores": [2, 3, 0, 2, 2, 2, 1, 2],
    "reactionTimes": [
        {"clickTimes": [1008, 381, 551, 266]},
        {"clickTimes": [1007, 999, 499, 245]},
        {"clickTimes": [897, 787, 300, 456]},
        {"clickTimes": [978, 654, 321, 111]},
        {"clickTimes": [700, 800, 600, 200]},
        {"clickTimes": [1500, 500, 700, 100]},
        {"clickTimes": [1400, 1300, 800, 900]},
        {"clickTimes": [1234, 5678, 4321, 8765]},
    ],
    "reactionTimesGlow": [
        [1348, 1961],
        [1546, 1460],
        [1423, 887, 1186],
        [1423, 111],
        [700],
        [3702],
        [1500, 1200],
        [900],
    ],
}



I need to know the max number of balls that can glow in a single game. In this example its 4

In [125]:
def process_reaction_times(data):
    processed_data = {
        "scores": data["scores"],
        "reactionTimes": [rt["clickTimes"] for rt in data["reactionTimes"]],
        "reactionTimesGlow": [
            glow + [pd.NA] * (4 - len(glow)) if len(glow) < 4 else glow[:4]
            for glow in data["reactionTimesGlow"]
        ],
    }
    df = pd.DataFrame(processed_data)

    # Flatten reactionTimes and reactionTimesGlow into separate columns
    reaction_df = pd.DataFrame(df["reactionTimes"].to_list(), columns=[
        f"reactionTime_{i+1}" for i in range(4)
    ])
    glow_df = pd.DataFrame(df["reactionTimesGlow"].to_list(), columns=[
        f"reactionTimesGlow_{i+1}" for i in range(4)
    ])
    return pd.concat([df.drop(columns=["reactionTimes", "reactionTimesGlow"]), reaction_df, glow_df], axis=1)

game_data = process_reaction_times(test_data)


If player doesn't click the glowing ball, we replace this with NA

In [126]:
game_data

,scores,reactionTime_1,reactionTime_2,reactionTime_3,reactionTime_4,reactionTimesGlow_1,reactionTimesGlow_2,reactionTimesGlow_3,reactionTimesGlow_4
0,2,1008,381,551,266,1348,1961,<NA>,<NA>
1,3,1007,999,499,245,1546,1460,<NA>,<NA>
2,0,897,787,300,456,1423,887,1186,<NA>
3,2,978,654,321,111,1423,111,<NA>,<NA>
4,2,700,800,600,200,700,<NA>,<NA>,<NA>
5,2,1500,500,700,100,3702,<NA>,<NA>,<NA>
6,1,1400,1300,800,900,1500,1200,<NA>,<NA>
7,2,1234,5678,4321,8765,900,<NA>,<NA>,<NA>


In [128]:
# Select the columns that represent reaction times
reaction_time_columns = [f"reactionTimesGlow_{i+1}" for i in range(4)]

# Calculate the average reaction time
game_data['averageReactionTimeGlow'] = game_data[reaction_time_columns].mean(axis=1)

# Convert the column to int64
game_data['averageReactionTimeGlow'] = game_data['averageReactionTimeGlow'].astype('int64')




Let's also calculate the mean reaction time for the blue balls 

In [129]:
# Select the columns that represent reaction times
reaction_time_columns = [f"reactionTime_{i+1}" for i in range(4)]

# Calculate the average reaction time
game_data['averageReactionTime'] = game_data[reaction_time_columns].mean(axis=1)

# Convert the column to int64
game_data['averageReactionTime'] = game_data['averageReactionTime'].astype('int64')




Find the mean of the reaction times for the glowing ball. Although this won't give us the data on how many balls the player has clicked, we know that 

In [130]:
game_data

,scores,reactionTime_1,reactionTime_2,reactionTime_3,reactionTime_4,reactionTimesGlow_1,reactionTimesGlow_2,reactionTimesGlow_3,reactionTimesGlow_4,averageReactionTimeGlow,averageReactionTime
0,2,1008,381,551,266,1348,1961,<NA>,<NA>,1654,551
1,3,1007,999,499,245,1546,1460,<NA>,<NA>,1503,687
2,0,897,787,300,456,1423,887,1186,<NA>,1165,610
3,2,978,654,321,111,1423,111,<NA>,<NA>,767,516
4,2,700,800,600,200,700,<NA>,<NA>,<NA>,700,575
5,2,1500,500,700,100,3702,<NA>,<NA>,<NA>,3702,700
6,1,1400,1300,800,900,1500,1200,<NA>,<NA>,1350,1100
7,2,1234,5678,4321,8765,900,<NA>,<NA>,<NA>,900,4999


In [131]:
game_data.dtypes

scores                      int64
reactionTime_1              int64
reactionTime_2              int64
reactionTime_3              int64
reactionTime_4              int64
reactionTimesGlow_1         int64
reactionTimesGlow_2        object
reactionTimesGlow_3        object
reactionTimesGlow_4        object
averageReactionTimeGlow     int64
averageReactionTime         int64
dtype: object

In [132]:
preprocessor = (
    create_pipeline()
    .add_stage(FormatData)
    .add_stage(ClassifyColumns)  # Classify numerical and other columns
)

descriptive_analysis = (
    create_pipeline()
    .add_stage(NumericalSummary)  # Analyze numerical columns
)

distribution_analysis = (create_pipeline())
intervariable_analysis = (create_pipeline().add_stage(Correlation))


analysis = (create_analysis()
            .add_preprocessor(preprocessor)
            .add_pipeline(descriptive_analysis)
            .add_pipeline(distribution_analysis)
            .add_pipeline(intervariable_analysis))


In [133]:
glow_game_analysis = analysis.run(game_data)

In [134]:
glow_game_analysis["summary"]["numerical"]


,scores,reactionTime_1,reactionTime_2,reactionTime_3,reactionTime_4,reactionTimesGlow_1,averageReactionTimeGlow,averageReactionTime
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,1.750000,1090.500000,1387.375000,1011.500000,1380.375000,1567.750000,1467.625000,1217.250000
std,0.886405,267.378811,1757.085894,1348.100569,2995.062171,914.108582,966.300595,1539.061472
min,0.000000,700.000000,381.000000,300.000000,100.000000,700.000000,700.000000,516.000000
25%,1.750000,957.750000,615.500000,454.500000,177.750000,1236.000000,866.750000,569.000000
50%,2.000000,1007.500000,793.500000,575.500000,255.500000,1423.000000,1257.500000,648.500000
75%,2.000000,1275.500000,1074.250000,725.000000,567.000000,1511.500000,1540.750000,800.000000
max,3.000000,1500.000000,5678.000000,4321.000000,8765.000000,3702.000000,3702.000000,4999.000000


In [135]:
glow_game_analysis["intervariable"]["correlation"]

,scores,reactionTime_1,reactionTime_2,reactionTime_3,reactionTime_4,reactionTimesGlow_1,averageReactionTimeGlow,averageReactionTime
scores,1.000000,-0.003617,0.082527,0.134134,0.064236,0.059151,0.126465,0.083930
reactionTime_1,-0.003617,1.000000,0.233769,0.292777,0.241905,0.678172,0.672309,0.291959
reactionTime_2,0.082527,0.233769,1.000000,0.985005,0.993658,-0.339822,-0.291358,0.994612
reactionTime_3,0.134134,0.292777,0.985005,1.000000,0.992616,-0.253652,-0.179704,0.995667
reactionTime_4,0.064236,0.241905,0.993658,0.992616,1.000000,-0.314108,-0.255132,0.997904
reactionTimesGlow_1,0.059151,0.678172,-0.339822,-0.253652,-0.314108,1.000000,0.958305,-0.275851
averageReactionTimeGlow,0.126465,0.672309,-0.291358,-0.179704,-0.255132,0.958305,1.000000,-0.217417
averageReactionTime,0.083930,0.291959,0.994612,0.995667,0.997904,-0.275851,-0.217417,1.000000
